# Action Analysis Module Demo

This notebook demonstrates the comprehensive action analysis capabilities of the AgentFarm analysis system using sample data from `docs/sample`.

## Overview

The action analysis module provides:
- Action frequency and success rate analysis
- Action sequence pattern detection
- Decision-making pattern analysis
- Reward and performance metrics
- Temporal action pattern analysis
- Comprehensive visualization capabilities

## Table of Contents
1. [Setup and Configuration](#setup)
2. [Loading Sample Data](#loading)
3. [Basic Action Analysis](#basic)
4. [Advanced Analysis Features](#advanced)
5. [Data Exploration](#exploration)
6. [Visualization Examples](#visualization)
7. [Custom Analysis Parameters](#custom)
8. [Summary and Insights](#summary)


## 1. Setup and Configuration {#setup}

First, let's set up the necessary imports and configuration for the analysis system.


In [1]:
# Standard library imports
import sys
import time
from pathlib import Path
import json

# Data analysis imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# AgentFarm imports
from farm.analysis.service import AnalysisService, AnalysisRequest
from farm.analysis.registry import get_module, get_module_names
from farm.core.services import EnvConfigService
from farm.analysis.actions import (
    compute_action_statistics,
    compute_sequence_patterns,
    compute_decision_patterns,
    compute_reward_metrics,
    plot_action_frequencies,
    plot_sequence_patterns,
    plot_decision_patterns,
    plot_reward_distributions
)
from farm.analysis.actions.data import process_action_data
from farm.analysis.common.context import AnalysisContext

print("✅ All imports successful!")
print(f"📁 Current working directory: {Path.cwd()}")
print(f"🐍 Python version: {sys.version}")


✅ All imports successful!
📁 Current working directory: /home/peril10/Dropbox/Development/AgentFarm/notebooks
🐍 Python version: 3.12.3 (main, Aug 14 2025, 17:47:21) [GCC 13.3.0]


## 2. Loading Sample Data {#loading}

Let's load and explore the sample data from the `docs/sample` directory.


In [2]:
# Set up paths - handle different working directories
current_dir = Path.cwd()
print(f"📁 Current working directory: {current_dir}")

# Try to find the project root and sample data
# If we're in notebooks/, go up one level to find docs/sample
if current_dir.name == "notebooks":
    project_root = current_dir.parent
    sample_path = project_root / "docs" / "sample"
    output_path = current_dir / "action_analysis_results"
else:
    # If we're in the project root
    sample_path = current_dir / "docs" / "sample"
    output_path = current_dir / "notebooks" / "action_analysis_results"

# Create output directory
output_path.mkdir(parents=True, exist_ok=True)

print(f"📂 Project root: {project_root if current_dir.name == 'notebooks' else current_dir}")
print(f"📂 Sample data path: {sample_path}")
print(f"📂 Output path: {output_path}")
print(f"📂 Sample path exists: {sample_path.exists()}")

# List contents of sample directory
if sample_path.exists():
    sample_files = list(sample_path.glob("*"))
    print(f"\n📄 Sample directory contents:")
    for file in sample_files:
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"   - {file.name} ({size_mb:.1f} MB)")
else:
    print("⚠️  Sample directory not found. Creating mock data for demonstration.")
    sample_path.mkdir(parents=True, exist_ok=True)


📁 Current working directory: /home/peril10/Dropbox/Development/AgentFarm/notebooks
📂 Project root: /home/peril10/Dropbox/Development/AgentFarm
📂 Sample data path: /home/peril10/Dropbox/Development/AgentFarm/docs/sample
📂 Output path: /home/peril10/Dropbox/Development/AgentFarm/notebooks/action_analysis_results
📂 Sample path exists: True

📄 Sample directory contents:
   - simulation.db-wal (0.0 MB)
   - simulation.db-shm (0.0 MB)
   - simulation.db (26.3 MB)


In [3]:
# First, let's check if we can connect to the database directly
print("🔍 Checking database connection...")

try:
    from farm.database.database import SimulationDatabase
    from farm.database.session_manager import SessionManager
    from farm.database.repositories.action_repository import ActionRepository
    
    # Try to connect to the database
    db_path = sample_path / "simulation.db"
    if db_path.exists():
        print(f"✅ Database file found: {db_path}")
        print(f"📊 Database size: {db_path.stat().st_size / (1024*1024):.1f} MB")
        
        # Try to connect and get basic info
        session_manager = SessionManager(f"sqlite:///{db_path}")
        repository = ActionRepository(session_manager)
        
        # Get total action count
        actions = repository.get_actions_by_scope("simulation")
        print(f"📈 Total actions in database: {len(actions)}")
        
        if actions:
            # Show sample action
            sample_action = actions[0]
            print(f"📋 Sample action: {sample_action.action_type} at step {sample_action.step_number}")
            print(f"🎯 Action types available: {set(a.action_type for a in actions[:100])}")
        
    else:
        print(f"❌ Database file not found: {db_path}")
        
except Exception as e:
    print(f"⚠️  Database connection failed: {e}")
    print("This is expected if the database structure is different or if there are import issues.")

print("\n" + "="*50)
print("🔄 Loading action data using the analysis module...")

try:
    # Use the action analysis data processor
    action_df = process_action_data(sample_path, use_database=True)
    
    print(f"✅ Successfully loaded action data!")
    print(f"📊 Data shape: {action_df.shape}")
    print(f"📋 Columns: {list(action_df.columns)}")
    
    # Display basic info about the data
    print("\n📈 Data Overview:")
    print(action_df.head())
    
    print("\n📊 Data Info:")
    print(action_df.info())
    
    print("\n📈 Basic Statistics:")
    print(action_df.describe())
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("\n🔄 Creating mock data for demonstration...")
    
    # Create mock action data for demonstration
    np.random.seed(42)
    n_steps = 100
    action_types = ['move', 'gather', 'attack', 'defend', 'rest']
    
    mock_data = []
    for step in range(n_steps):
        for action_type in action_types:
            # Simulate varying frequencies over time
            base_freq = np.random.poisson(5)
            if action_type == 'move':
                base_freq += int(step * 0.1)  # Increasing movement over time
            elif action_type == 'attack':
                base_freq += int(np.sin(step * 0.1) * 3)  # Cyclical attacks
            
            mock_data.append({
                'step': step,
                'action_type': action_type,
                'frequency': max(0, base_freq)
            })
    
    action_df = pd.DataFrame(mock_data)
    print(f"✅ Created mock data with shape: {action_df.shape}")
    print("\n📈 Mock Data Sample:")
    print(action_df.head(10))


🔍 Checking database connection...
✅ Database file found: /home/peril10/Dropbox/Development/AgentFarm/docs/sample/simulation.db
📊 Database size: 26.3 MB
📈 Total actions in database: 46205
📋 Sample action: attack at step 0
🎯 Action types available: {'gather', 'attack', 'move', 'pass', 'reproduce', 'defend', 'share'}

🔄 Loading action data using the analysis module...
2025-10-05 17:15:50 [debug    ] Found database at: /home/peril10/Dropbox/Development/AgentFarm/docs/sample/simulation.db
2025-10-05 17:15:50 [info     ] Loading actions from database: /home/peril10/Dropbox/Development/AgentFarm/docs/sample/simulation.db
2025-10-05 17:15:51 [info     ] Loaded 46205 action records from database
2025-10-05 17:15:51 [info     ] Aggregated into 6960 (step, action_type) frequency records
✅ Successfully loaded action data!
📊 Data shape: (6960, 3)
📋 Columns: ['step', 'action_type', 'frequency']

📈 Data Overview:
   step action_type  frequency
0     0      attack          3
1     0      defend       

## 3. Basic Action Analysis {#basic}

Now let's run a basic action analysis using the AnalysisService. This will demonstrate the core functionality of the action analysis module.


In [4]:
# Initialize the analysis service
print("🔧 Initializing Analysis Service...")
config_service = EnvConfigService()
service = AnalysisService(config_service)

# List available modules to confirm actions module is available
available_modules = get_module_names()
print(f"📋 Available analysis modules: {available_modules}")

if 'actions' in available_modules:
    print("✅ Actions module is available!")
    
    # Get detailed info about the actions module
    actions_module = get_module('actions')
    print(f"📝 Module description: {actions_module.description}")
    print(f"🔧 Available function groups: {list(actions_module._groups.keys())}")
    print(f"⚙️  Available functions: {list(actions_module._functions.keys())}")
else:
    print("❌ Actions module not found in available modules")


🔧 Initializing Analysis Service...
2025-10-05 17:15:51 [info     ] No modules configured, attempting to register built-in modules
2025-10-05 17:15:51 [info     ] Registered analysis module: genesis
2025-10-05 17:15:51 [info     ] Registered analysis module: dominance
2025-10-05 17:15:51 [info     ] Registered analysis module: advantage
2025-10-05 17:15:51 [info     ] Registered analysis module: agents
2025-10-05 17:15:51 [info     ] Registered analysis module: learning
2025-10-05 17:15:51 [info     ] Registered analysis module: social_behavior
2025-10-05 17:15:51 [info     ] Registered analysis module: spatial
2025-10-05 17:15:51 [info     ] Registered analysis module: temporal
2025-10-05 17:15:51 [info     ] Registered analysis module: resources
2025-10-05 17:15:51 [info     ] Registered analysis module: actions
2025-10-05 17:15:51 [info     ] Registered analysis module: combat
2025-10-05 17:15:51 [info     ] Registered analysis module: significant_events
2025-10-05 17:15:51 [info    

In [5]:
# Create a basic analysis request
print("📋 Creating basic analysis request...")

basic_request = AnalysisRequest(
    module_name="actions",
    experiment_path=sample_path,
    output_path=output_path / "basic_analysis",
    group="basic",  # Run basic analysis (stats + basic plots)
    enable_caching=True
)

print(f"📁 Output will be saved to: {basic_request.output_path}")
print(f"🎯 Analysis group: {basic_request.group}")
print(f"💾 Caching enabled: {basic_request.enable_caching}")


📋 Creating basic analysis request...
📁 Output will be saved to: /home/peril10/Dropbox/Development/AgentFarm/notebooks/action_analysis_results/basic_analysis
🎯 Analysis group: basic
💾 Caching enabled: True


In [6]:
# Run the basic analysis
print("🚀 Running basic action analysis...")
start_time = time.time()

try:
    basic_result = service.run(basic_request)
    execution_time = time.time() - start_time
    
    if basic_result.success:
        print(f"✅ Analysis completed successfully!")
        print(f"⏱️  Execution time: {execution_time:.2f}s")
        print(f"📁 Results saved to: {basic_result.output_path}")
        print(f"💾 Cache hit: {basic_result.cache_hit}")
        
        # List generated files
        if basic_result.output_path.exists():
            files = list(basic_result.output_path.glob("*"))
            print(f"\n📄 Generated files ({len(files)}):")
            for file in sorted(files):
                size = file.stat().st_size
                print(f"   - {file.name} ({size} bytes)")
    else:
        print(f"❌ Analysis failed: {basic_result.error}")
        
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    import traceback
    traceback.print_exc()


🚀 Running basic action analysis...
2025-10-05 17:15:51 [info     ] Loaded result from cache: 891afe6aeb871377
2025-10-05 17:15:51 [info     ] Using cached result for actions
✅ Analysis completed successfully!
⏱️  Execution time: 0.00s
📁 Results saved to: /home/peril10/Dropbox/Development/AgentFarm/notebooks/action_analysis_results/basic_analysis
💾 Cache hit: True

📄 Generated files (4):
   - action_distribution.png (485745 bytes)
   - action_patterns.csv (88964 bytes)
   - action_statistics.json (2216 bytes)
   - analysis_summary.json (347 bytes)


## 4. Advanced Analysis Features {#advanced}

Let's explore more advanced analysis features including sequence patterns, decision analysis, and reward metrics.


In [7]:
# Run comprehensive analysis with all functions
print("🔬 Running comprehensive action analysis...")

comprehensive_request = AnalysisRequest(
    module_name="actions",
    experiment_path=sample_path,
    output_path=output_path / "comprehensive_analysis",
    group="all",  # Run all available functions
    enable_caching=True
)

start_time = time.time()
comprehensive_result = service.run(comprehensive_request)
execution_time = time.time() - start_time

if comprehensive_result.success:
    print(f"✅ Comprehensive analysis completed!")
    print(f"⏱️  Execution time: {execution_time:.2f}s")
    
    # List all generated files
    if comprehensive_result.output_path.exists():
        files = list(comprehensive_result.output_path.glob("*"))
        print(f"\n📄 Generated files ({len(files)}):")
        for file in sorted(files):
            size = file.stat().st_size
            print(f"   - {file.name} ({size} bytes)")
else:
    print(f"❌ Comprehensive analysis failed: {comprehensive_result.error}")


🔬 Running comprehensive action analysis...
2025-10-05 17:15:51 [info     ] Loaded result from cache: a73a3e2b855c1920
2025-10-05 17:15:51 [info     ] Using cached result for actions
✅ Comprehensive analysis completed!
⏱️  Execution time: 0.00s

📄 Generated files (7):
   - action_distribution.png (485745 bytes)
   - action_patterns.csv (88964 bytes)
   - action_statistics.json (2216 bytes)
   - analysis_summary.json (356 bytes)
   - decision_patterns.json (2 bytes)
   - reward_analysis.json (2 bytes)
   - sequence_patterns.json (2 bytes)


In [8]:
# Run analysis with specific function groups
print("🎯 Running analysis with specific function groups...")

# Analysis-only group (no plots)
analysis_request = AnalysisRequest(
    module_name="actions",
    experiment_path=sample_path,
    output_path=output_path / "analysis_only",
    group="analysis",  # Only analysis functions, no plots
    enable_caching=True
)

analysis_result = service.run(analysis_request)

if analysis_result.success:
    print(f"✅ Analysis-only completed!")
    
    # Load and display some results
    results_path = analysis_result.output_path
    
    # Try to load action statistics
    stats_file = results_path / "action_statistics.json"
    if stats_file.exists():
        with open(stats_file, 'r') as f:
            stats = json.load(f)
        
        print("\n📊 Action Statistics Summary:")
        if 'most_common_action' in stats:
            print(f"   Most common action: {stats['most_common_action']}")
            print(f"   Most common frequency: {stats['most_common_frequency']:.2f}")
        
        if 'action_types' in stats:
            print(f"   Number of action types: {len(stats['action_types'])}")
            for action_type, type_stats in stats['action_types'].items():
                if 'frequency' in type_stats and 'mean' in type_stats['frequency']:
                    print(f"   - {action_type}: avg frequency = {type_stats['frequency']['mean']:.2f}")
else:
    print(f"❌ Analysis-only failed: {analysis_result.error}")


🎯 Running analysis with specific function groups...
2025-10-05 17:15:51 [info     ] Loaded result from cache: 57e4f40b7e6bbb36
2025-10-05 17:15:51 [info     ] Using cached result for actions
✅ Analysis-only completed!

📊 Action Statistics Summary:
   Most common action: pass
   Most common frequency: 6.75
   Number of action types: 7
   - attack: avg frequency = 6.72
   - defend: avg frequency = 6.64
   - gather: avg frequency = 6.59
   - move: avg frequency = 6.55
   - pass: avg frequency = 6.75
   - reproduce: avg frequency = 6.55
   - share: avg frequency = 6.68


## 5. Data Exploration {#exploration}

Let's explore the raw action data and compute some statistics directly to understand what the analysis module is working with.


In [9]:
# Explore the action data in detail
print("🔍 Exploring action data...")
print(f"📊 Dataset shape: {action_df.shape}")
print(f"📋 Columns: {list(action_df.columns)}")

# Check for missing values
print("\n❓ Missing values:")
missing = action_df.isnull().sum()
for col, count in missing.items():
    if count > 0:
        print(f"   {col}: {count} missing values")
    else:
        print(f"   {col}: No missing values")

# Unique action types
if 'action_type' in action_df.columns:
    unique_actions = action_df['action_type'].unique()
    print(f"\n🎯 Unique action types ({len(unique_actions)}): {list(unique_actions)}")

# Step range
if 'step' in action_df.columns:
    step_range = (action_df['step'].min(), action_df['step'].max())
    print(f"\n📈 Step range: {step_range[0]} to {step_range[1]}")

# Frequency statistics
if 'frequency' in action_df.columns:
    freq_stats = action_df['frequency'].describe()
    print(f"\n📊 Frequency statistics:")
    print(freq_stats)


🔍 Exploring action data...
📊 Dataset shape: (6960, 3)
📋 Columns: ['step', 'action_type', 'frequency']

❓ Missing values:
   step: No missing values
   action_type: No missing values
   frequency: No missing values

🎯 Unique action types (7): ['attack', 'defend', 'gather', 'move', 'pass', 'reproduce', 'share']

📈 Step range: 0 to 999

📊 Frequency statistics:
count    6960.000000
mean        6.638649
std         4.184016
min         1.000000
25%         4.000000
50%         6.000000
75%         8.000000
max        28.000000
Name: frequency, dtype: float64


In [10]:
# Compute action statistics directly using the compute functions
print("🧮 Computing action statistics directly...")

try:
    # Compute basic action statistics
    action_stats = compute_action_statistics(action_df)
    
    print("✅ Action statistics computed successfully!")
    print("\n📊 Action Statistics:")
    
    # Display key statistics
    if 'most_common_action' in action_stats:
        print(f"   Most common action: {action_stats['most_common_action']}")
        print(f"   Most common frequency: {action_stats['most_common_frequency']:.2f}")
    
    if 'total_actions' in action_stats:
        total_stats = action_stats['total_actions']
        print(f"   Total actions - Mean: {total_stats.get('mean', 'N/A'):.2f}")
        print(f"   Total actions - Std: {total_stats.get('std', 'N/A'):.2f}")
    
    if 'action_types' in action_stats:
        print(f"\n   Action type details:")
        for action_type, type_stats in action_stats['action_types'].items():
            if 'frequency' in type_stats and 'mean' in type_stats['frequency']:
                mean_freq = type_stats['frequency']['mean']
                std_freq = type_stats['frequency'].get('std', 0)
                print(f"     {action_type}: {mean_freq:.2f} ± {std_freq:.2f}")
    
except Exception as e:
    print(f"❌ Error computing statistics: {e}")
    import traceback
    traceback.print_exc()


🧮 Computing action statistics directly...
✅ Action statistics computed successfully!

📊 Action Statistics:
   Most common action: pass
   Most common frequency: 6.75
   Total actions - Mean: 46.20
   Total actions - Std: 24.29

   Action type details:
     attack: 6.72 ± 4.19
     defend: 6.64 ± 4.24
     gather: 6.59 ± 4.14
     move: 6.55 ± 4.03
     pass: 6.75 ± 4.30
     reproduce: 6.55 ± 4.12
     share: 6.68 ± 4.25


In [11]:
# Create a simple visualization of the action data
print("📊 Creating action data visualization...")

if not action_df.empty and 'action_type' in action_df.columns:
    # Create a figure with multiple subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Action Data Overview', fontsize=16, fontweight='bold')
    
    # 1. Action frequency over time
    if 'step' in action_df.columns and 'frequency' in action_df.columns:
        ax1 = axes[0, 0]
        for action_type in action_df['action_type'].unique():
            type_data = action_df[action_df['action_type'] == action_type]
            ax1.plot(type_data['step'], type_data['frequency'], 
                    label=action_type, linewidth=2, marker='o', markersize=3)
        ax1.set_xlabel('Simulation Step')
        ax1.set_ylabel('Action Frequency')
        ax1.set_title('Action Frequencies Over Time')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
    
    # 2. Total actions per step
    if 'step' in action_df.columns and 'frequency' in action_df.columns:
        ax2 = axes[0, 1]
        total_actions = action_df.groupby('step')['frequency'].sum()
        ax2.plot(total_actions.index, total_actions.values, 
                linewidth=2, color='red', marker='s', markersize=4)
        ax2.set_xlabel('Simulation Step')
        ax2.set_ylabel('Total Actions')
        ax2.set_title('Total Actions Per Step')
        ax2.grid(True, alpha=0.3)
    
    # 3. Action type distribution
    ax3 = axes[1, 0]
    if 'frequency' in action_df.columns:
        action_totals = action_df.groupby('action_type')['frequency'].sum()
    else:
        action_totals = action_df['action_type'].value_counts()
    
    action_totals.plot(kind='bar', ax=ax3, color='skyblue', edgecolor='black')
    ax3.set_xlabel('Action Type')
    ax3.set_ylabel('Total Frequency')
    ax3.set_title('Action Type Distribution')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(True, alpha=0.3)
    
    # 4. Action diversity over time
    if 'step' in action_df.columns:
        ax4 = axes[1, 1]
        diversity = action_df.groupby('step')['action_type'].nunique()
        ax4.plot(diversity.index, diversity.values, 
                linewidth=2, color='green', marker='^', markersize=4)
        ax4.set_xlabel('Simulation Step')
        ax4.set_ylabel('Number of Action Types')
        ax4.set_title('Action Diversity Over Time')
        ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualization created successfully!")
else:
    print("⚠️  Cannot create visualization - missing required columns")


📊 Creating action data visualization...
✅ Visualization created successfully!


/tmp/ipykernel_68744/3217843017.py:59: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 6. Visualization Examples {#visualization}

Let's demonstrate the plotting capabilities of the action analysis module by running the plotting functions directly.


In [12]:
# Run plotting-only analysis
print("🎨 Running plotting analysis...")

plotting_request = AnalysisRequest(
    module_name="actions",
    experiment_path=sample_path,
    output_path=output_path / "plotting_analysis",
    group="plots",  # Only plotting functions
    enable_caching=True
)

plotting_result = service.run(plotting_request)

if plotting_result.success:
    print(f"✅ Plotting analysis completed!")
    
    # List generated plot files
    if plotting_result.output_path.exists():
        plot_files = list(plotting_result.output_path.glob("*.png"))
        print(f"\n🖼️  Generated plots ({len(plot_files)}):")
        for plot_file in sorted(plot_files):
            size = plot_file.stat().st_size
            print(f"   - {plot_file.name} ({size} bytes)")
else:
    print(f"❌ Plotting analysis failed: {plotting_result.error}")


🎨 Running plotting analysis...
2025-10-05 17:15:51 [info     ] Loaded result from cache: 434c623c8847b159
2025-10-05 17:15:51 [info     ] Using cached result for actions
✅ Plotting analysis completed!

🖼️  Generated plots (1):
   - action_distribution.png (485745 bytes)


In [13]:
# Demonstrate direct plotting functions
print("🎨 Demonstrating direct plotting functions...")

# Create a mock analysis context for plotting
class MockAnalysisContext:
    def __init__(self, output_dir):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # Create a proper mock logger that accepts the message parameter
        class MockLogger:
            def info(self, msg):
                print(f"INFO: {msg}")
            
            def warning(self, msg):
                print(f"WARNING: {msg}")
            
            def error(self, msg):
                print(f"ERROR: {msg}")
        
        self.logger = MockLogger()
    
    def get_output_file(self, filename):
        return self.output_dir / filename
    
    def report_progress(self, message, progress):
        print(f"Progress [{progress:.1%}]: {message}")

# Create mock context
mock_context = MockAnalysisContext(output_path / "direct_plots")

try:
    # Plot action frequencies
    print("\n📊 Plotting action frequencies...")
    plot_action_frequencies(action_df, mock_context, figsize=(12, 6), dpi=150)
    
    # Plot action distribution
    print("\n📊 Plotting action distribution...")
    from farm.analysis.actions.plot import plot_action_distribution
    plot_action_distribution(action_df, mock_context, figsize=(10, 6), dpi=150)
    
    print("✅ Direct plotting completed!")
    
    # List generated plots
    plot_files = list(mock_context.output_dir.glob("*.png"))
    print(f"\n🖼️  Generated plots ({len(plot_files)}):")
    for plot_file in sorted(plot_files):
        size = plot_file.stat().st_size
        print(f"   - {plot_file.name} ({size} bytes)")
        
except Exception as e:
    print(f"❌ Error in direct plotting: {e}")
    import traceback
    traceback.print_exc()


🎨 Demonstrating direct plotting functions...

📊 Plotting action frequencies...
INFO: Creating action frequencies plot...
INFO: Saved plot to /home/peril10/Dropbox/Development/AgentFarm/notebooks/action_analysis_results/direct_plots/action_distribution.png

📊 Plotting action distribution...
INFO: Creating action distribution plot...
INFO: Saved plot to /home/peril10/Dropbox/Development/AgentFarm/notebooks/action_analysis_results/direct_plots/action_distribution.png
✅ Direct plotting completed!

🖼️  Generated plots (1):
   - action_distribution.png (47609 bytes)


## 7. Custom Analysis Parameters {#custom}

Let's demonstrate how to use custom parameters for analysis and plotting.


In [14]:
# Run analysis with custom parameters
print("⚙️  Running analysis with custom parameters...")

# Define progress callback
def progress_callback(message: str, progress: float):
    print(f"[{progress:>5.1%}] {message}")

# Create request with custom parameters
custom_request = AnalysisRequest(
    module_name="actions",
    experiment_path=sample_path,
    output_path=output_path / "custom_analysis",
    group="all",
    progress_callback=progress_callback,
    analysis_kwargs={
        "plot_frequencies": {
            "figsize": (15, 8),
            "dpi": 300,
            "colors": ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]
        },
        "plot_action_distribution": {
            "figsize": (12, 8),
            "dpi": 300
        }
    },
    metadata={
        "analysis_type": "custom_demo",
        "notebook_version": "1.0",
        "custom_parameters": True
    }
)

print(f"📋 Custom parameters configured:")
print(f"   - Progress callback: {'Yes' if custom_request.progress_callback else 'No'}")
print(f"   - Analysis kwargs: {len(custom_request.analysis_kwargs)} functions")
print(f"   - Metadata: {len(custom_request.metadata)} items")

# Run the custom analysis
start_time = time.time()
custom_result = service.run(custom_request)
execution_time = time.time() - start_time

if custom_result.success:
    print(f"\n✅ Custom analysis completed!")
    print(f"⏱️  Execution time: {execution_time:.2f}s")
    print(f"📁 Results: {custom_result.output_path}")
    print(f"📊 Metadata: {custom_result.metadata}")
else:
    print(f"❌ Custom analysis failed: {custom_result.error}")


⚙️  Running analysis with custom parameters...
📋 Custom parameters configured:
   - Progress callback: Yes
   - Analysis kwargs: 2 functions
   - Metadata: 3 items
2025-10-05 17:15:52 [info     ] Loaded result from cache: f28859da4597aba9
2025-10-05 17:15:52 [info     ] Using cached result for actions

✅ Custom analysis completed!
⏱️  Execution time: 0.00s
📁 Results: /home/peril10/Dropbox/Development/AgentFarm/notebooks/action_analysis_results/custom_analysis
📊 Metadata: {'analysis_type': 'custom_demo', 'notebook_version': '1.0', 'custom_parameters': True}


## 8. Summary and Insights {#summary}

Let's summarize what we've learned about the action analysis module and the insights we can derive from the data.


In [15]:
# Summary of analysis results
print("📋 Action Analysis Module Summary")
print("=" * 50)

print("\n🔧 Module Capabilities:")
print("   ✅ Action frequency analysis")
print("   ✅ Action sequence pattern detection")
print("   ✅ Decision-making pattern analysis")
print("   ✅ Reward and performance metrics")
print("   ✅ Comprehensive visualization")
print("   ✅ Custom parameter configuration")
print("   ✅ Caching for performance")
print("   ✅ Progress tracking")

print("\n📊 Data Insights:")
if not action_df.empty:
    print(f"   📈 Total data points: {len(action_df)}")
    
    if 'action_type' in action_df.columns:
        unique_actions = action_df['action_type'].nunique()
        print(f"   🎯 Unique action types: {unique_actions}")
        
        # Most common action
        if 'frequency' in action_df.columns:
            most_common = action_df.groupby('action_type')['frequency'].sum().idxmax()
            most_common_count = action_df.groupby('action_type')['frequency'].sum().max()
            print(f"   🏆 Most common action: {most_common} ({most_common_count} total)")
    
    if 'step' in action_df.columns:
        step_range = (action_df['step'].min(), action_df['step'].max())
        print(f"   ⏰ Simulation duration: {step_range[1] - step_range[0] + 1} steps")
        
        # Action diversity trend
        if 'action_type' in action_df.columns:
            diversity_by_step = action_df.groupby('step')['action_type'].nunique()
            avg_diversity = diversity_by_step.mean()
            print(f"   🎭 Average action diversity: {avg_diversity:.2f} types per step")

print("\n🎯 Analysis Results Summary:")
analyses_run = [
    ("Basic Analysis", basic_result.success if 'basic_result' in locals() else False),
    ("Comprehensive Analysis", comprehensive_result.success if 'comprehensive_result' in locals() else False),
    ("Analysis Only", analysis_result.success if 'analysis_result' in locals() else False),
    ("Plotting Analysis", plotting_result.success if 'plotting_result' in locals() else False),
    ("Custom Analysis", custom_result.success if 'custom_result' in locals() else False)
]

for analysis_name, success in analyses_run:
    status = "✅" if success else "❌"
    print(f"   {status} {analysis_name}")

print("\n📁 Output Files Generated:")
total_files = 0
for result_name in ['basic_result', 'comprehensive_result', 'analysis_result', 'plotting_result', 'custom_result']:
    if result_name in locals():
        result = locals()[result_name]
        if result.success and result.output_path.exists():
            files = list(result.output_path.glob("*"))
            total_files += len(files)
            print(f"   📂 {result_name.replace('_result', '')}: {len(files)} files")

print(f"\n📊 Total files generated: {total_files}")

print("\n🚀 Key Takeaways:")
print("   • The action analysis module provides comprehensive analysis capabilities")
print("   • Multiple analysis groups allow for flexible analysis depth")
print("   • Custom parameters enable fine-tuned control over analysis and visualization")
print("   • Caching improves performance for repeated analyses")
print("   • Progress tracking provides real-time feedback")
print("   • Results are automatically saved in organized output directories")
print("   • The module handles both real and mock data gracefully")

print("\n🎉 Action Analysis Demo Complete!")
print("=" * 50)


📋 Action Analysis Module Summary

🔧 Module Capabilities:
   ✅ Action frequency analysis
   ✅ Action sequence pattern detection
   ✅ Decision-making pattern analysis
   ✅ Reward and performance metrics
   ✅ Comprehensive visualization
   ✅ Custom parameter configuration
   ✅ Caching for performance
   ✅ Progress tracking

📊 Data Insights:
   📈 Total data points: 6960
   🎯 Unique action types: 7
   🏆 Most common action: pass (6697 total)
   ⏰ Simulation duration: 1000 steps
   🎭 Average action diversity: 6.96 types per step

🎯 Analysis Results Summary:
   ✅ Basic Analysis
   ✅ Comprehensive Analysis
   ✅ Analysis Only
   ✅ Plotting Analysis
   ✅ Custom Analysis

📁 Output Files Generated:
   📂 basic: 4 files
   📂 comprehensive: 7 files
   📂 analysis: 6 files
   📂 plotting: 2 files
   📂 custom: 7 files

📊 Total files generated: 26

🚀 Key Takeaways:
   • The action analysis module provides comprehensive analysis capabilities
   • Multiple analysis groups allow for flexible analysis depth
  

## Next Steps

This notebook has demonstrated the core capabilities of the action analysis module. Here are some suggested next steps:

1. **Explore Other Analysis Modules**: Try the population, resources, agents, or learning analysis modules
2. **Custom Analysis Functions**: Create your own analysis functions following the module pattern
3. **Batch Analysis**: Run analysis on multiple experiments simultaneously
4. **Integration**: Integrate the analysis results into your simulation workflow
5. **Advanced Visualization**: Create custom visualizations using the computed statistics

For more information, see:
- `farm/analysis/README.md` - Complete documentation
- `examples/analysis_example.py` - Additional examples
- `farm/analysis/actions/` - Action analysis module source code
